In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import datetime as dt


In [ ]:
inverter1 = pd.read_excel(r"C:\Users\golu6\OneDrive\Documents\My Data Sources\inverter1.xlsx")
inverter2 = pd.read_excel(r"C:\Users\golu6\OneDrive\Documents\My Data Sources\inverter2.xlsx")
inverter3 = pd.read_excel(r"C:\Users\golu6\OneDrive\Documents\My Data Sources\inverter3.xlsx")
weather = pd.read_excel(r"C:\Users\golu6\OneDrive\Documents\My Data Sources\weather.xlsx")

In [ ]:
inverter1.head()

In [ ]:
inverter2.head()

In [ ]:
inverter3.head()

In [ ]:
weather.head()

In [ ]:
inverter1.isnull().sum()

In [ ]:
inverter2.isnull().sum()

In [ ]:
inverter3.isnull().sum()

In [ ]:
weather.isnull().sum()

In [ ]:
inverter3.describe

In [ ]:
inverter2.describe

In [ ]:
inverter1.describe

In [ ]:
weather.describe

In [ ]:
inverter1.shape

In [ ]:
inverter2.shape

In [ ]:
inverter3.shape

In [ ]:
weather.shape

In [ ]:
inverter1.describe().T

In [ ]:
inverter1.duplicated().sum()

In [ ]:
weather.describe().T

In [ ]:
inverter2.duplicated().sum()

In [ ]:
inverter1 = inverter1[inverter1['DATE & TIME'].str.contains(r'\d{2}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', na=False)]
inverter1['DATE & TIME'] = pd.to_datetime(inverter1['DATE & TIME'], format='%d-%m-%y %H:%M:%S')


In [ ]:
inverter2 = inverter2[inverter2['DATE & TIME'].str.contains(r'\d{2}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', na=False)]
inverter2['DATE & TIME'] = pd.to_datetime(inverter2['DATE & TIME'], format='%d-%m-%y %H:%M:%S')

In [ ]:
inverter3 = inverter3[inverter3['DATE & TIME'].str.contains(r'\d{2}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', na=False)]
inverter3['DATE & TIME'] = pd.to_datetime(inverter3['DATE & TIME'], format='%d-%m-%y %H:%M:%S')

In [ ]:
weather['DATE & TIME'] = pd.to_datetime(weather['DATE & TIME'], format='%d-%m-%y %H:%M:%S')


In [ ]:
merged_inverter = pd.merge(inverter1, inverter2, on='DATE & TIME', how='outer')

merged_inverter = pd.merge(merged_inverter, inverter3, on='DATE & TIME', how='outer')

merged_inverter = merged_inverter.sort_values('DATE & TIME')


In [ ]:
merged_all= weather.merge(merged_inverter, on='DATE & TIME', how='left')

In [ ]:
merged_all

In [ ]:
merged_all.info()

In [ ]:
merged_all.isnull().sum()

In [ ]:
merged_all.columns = merged_all.columns.str.strip()

columns_to_fill = [ 
    "DC CURRENT_x", "DC VOLTAGE_x", "DC POWER_x", "TEMPERATURE_x",
    "DC CURRENT_y", "DC VOLTAGE_y", "DC POWER_y", "TEMPERATURE_y",
    "DC CURRENT", "DC VOLTAGE", "DC POWER", "TEMPERATURE"
]

# Fill NaN values safely
for col in columns_to_fill:
    if col in merged_all.columns:  # Make sure column exists
        if merged_all[col].notna().any():
            merged_all[col] = merged_all[col].fillna(merged_all[col].median()).infer_objects(copy=False)
        else:
            merged_all[col] = merged_all[col].fillna(0)


In [ ]:
merged_all

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
plt.figure(figsize=(8, 5))
plt.scatter(merged_all['DC POWER_x'], merged_all['DC CURRENT_y'])
plt.title('DC vs DC CURRENT_y')
plt.xlabel('DC POWER_x (kW)')
plt.ylabel('DC CURRENT_y(kW)')
plt.grid(True)
plt.show()


In [ ]:
Q1 = merged_all['DC POWER_x'].quantile(0.25)
Q3 = merged_all['DC POWER_x'].quantile(0.75)
IQR = Q3 - Q1
lower_bound=Q1-1.5*IQR
upper_bound=Q3+1.5*IQR

outliers = merged_all[merged_all['DC POWER_x'] > upper_bound]

print(f"Upper Bound: {upper_bound}")
print(f"Outliers: \n{outliers}")

In [ ]:
#HISTOGRAM FOR UNDERSTAND DISTRIBUTION
for i in merged_all.select_dtypes(include="number").columns:
    sns.histplot(data=merged_all , x=i)
    plt.show()

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(inverter1['DATE & TIME'], inverter1['DC POWER'], color='blue', linewidth=1)
plt.title('DC Power Output Over Time - inverter1')
plt.xlabel('Time')
plt.ylabel('DC Power (kW)')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
#BOXPLOT FOR FINDING OUTLIERS
for i in merged_all.select_dtypes(include="number").columns:
    sns.boxplot(data=merged_all , x=i)
    plt.show()

In [ ]:
# FIRST MOMENT BUSINESS DECISION
merged_all.mean()

In [ ]:
merged_all.median()

In [ ]:
merged_all.mode()

In [ ]:
merged_all.to_excel("EDA.xlsx" , index=f)